In [1]:
import requests
from bs4 import BeautifulSoup
import chardet
import re 
import xlwt
import time
import random

In [2]:
def getHtml(index):
    USER_AGENTS=["Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; \
        .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1;\
         .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko)\
         Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) \
        Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732;\
         .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; \
        SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; \
        Media Center PC 6.0; .NET4.0C; .NET4.0E)"]
    proxies=[{"HTTP":"117.63.78.64:6666"},
               {"HTTPS":"114.225.169.215:53128"},
               {"HTTPS":"222.185.22.108:6666"}]
   # ,proxies=random.choice(proxies)
    print("正在抓取第",index+1,"页面信息")
    url="https://movie.douban.com/top250?start='+str(index*25)+'&filter="
    r=requests.get(url,headers={"User-Agent":random.choice(USER_AGENTS)})
    code = chardet.detect(r.content)['encoding']
    return r.content.decode(code)


In [3]:
reg= re.compile('.*(\d{4}).*')
def getData(n):
    datalist=[]
    for step in range(n):
        global reg
        time.sleep(0.2)
        html=getHtml(step)
        soup=BeautifulSoup(html,"html.parser")
        parent = soup.find('div',attrs={'id':'content'}) #父节点
        lis = parent.find_all('li')
        
        for li in lis:
            data = []
            film_name=li.find('div',attrs={'class':'hd'}).find('span').get_text()
            data.append(film_name)
            film_time_str=li.find('div',attrs={'class':'bd'}).find('p').get_text()
            film_time=re.findall(reg,film_time_str)[0]
            data.append(film_time)
            film_score=li.find('div',attrs={'class':'star'}).find_all('span')[1].get_text()
            data.append(film_score)
            person_number=li.find('div',attrs={'class':'star'}).find_all('span')[3].get_text()
            number= re.findall(re.compile('\d*'),person_number)[0]
            data.append(number)
            #获取简评 ，因为个别没有标签，加以判断
            if li.find('div',attrs={'class':'bd'}).find('p',attrs={'class':'quote'}):
                evaluate = li.find('div',attrs={'class':'bd'}).find('p',attrs={'class':'quote'}).find('span').get_text()
            else:
                evaluate= ''
            data.append(evaluate)
            datalist.append(data)
    return datalist

In [4]:
def SavetoExcel(n,fileName):
    book=xlwt.Workbook()
    sheet =book.add_sheet('豆瓣电影')
    data=getData(n)
    col=('电影名称','上映时间','电影评分','评分人数','电影简评')
    for k,v in enumerate(col):
        sheet.write(0,k,v)
    for i,each in enumerate(data):
        for j,value in enumerate(each):
            sheet.write(i+1,j,value)
    book.save(fileName)
    

In [5]:
SavetoExcel(10,'豆瓣.xls')
print('结束')

正在抓取第 1 页面信息
正在抓取第 2 页面信息
正在抓取第 3 页面信息
正在抓取第 4 页面信息
正在抓取第 5 页面信息
正在抓取第 6 页面信息
正在抓取第 7 页面信息
正在抓取第 8 页面信息
正在抓取第 9 页面信息
正在抓取第 10 页面信息
结束
